# Dokumentation für meine Implementation des Papers von Sebastian

Hier möchte ich die importierten Funktionalitäten beschreiben, die sich primär aus dem Paper "Least squares approximate policy iteration for learning bid prices in choice-based revenue management" von Sebastian Koch ergeben.

Das Dokument dient rein der Dokumentation. Die jeweils aktuellste Implementierung findet sich in *func_Koch.py*.

## Vorbemerkungen

Es gibt in meinem Modell:
* i = 0, ..., m-1 Ressourcen
* j = 0, ..., n-1 Produkte
* l = 0, ..., L-1 Kundensegmente
* t = 0, ..., T-1 Zeiten

Ich verwende folgende Daten als **Inputvariablen**:

| Name | Datentyp | Beschreibung |
| :--- | --- | :--- |
| **resources** | np.array | Ressourcen von 0 bis m-1 |
| capacities | np.array | Kapazitäten für alle Ressourcen |
| **products** | np.array | Produkte von 0 bis n-1 |
| revenues | np.array | Revenues für alle Produkte |
| A | np.array | Kapazitätsbedarfsmatrix mit m Zeilen und n Spalten: a_{ij} = 1 wenn Ressource i für Produkt j gebraucht wird |
| **customer_segments** | np.array | Kundensgemente von 0 bis L-1 |
| preference_weights | np.array | Matrix mit L Zeilen: jede Zeile l enthält die Präferenzen von Kundensegment l für alle Produkte |
| preference_no_purchase | np.array | Nichtkauf-Präferenzen für alle Kundensegmente |
| arrival_probabilities | np.array | Vektor mit Ankunftswahrscheinlichkeiten für alle Kundensegmente |
| **times** | np.array | Zeiten von 0 bis T-1 |

## Beispieldaten

Um die Korrektheit der nachstehenden Methoden überprüfen zu können, möchte ich hier den Beispieldatensatz aus der Sektion 4.2. von Koch verwenden (single-leg flight example).

In [1]:
import numpy as np

example = "singleLegFlight"

if example == "singleLegFlight":
    n = 4
    products = np.arange(n)
    revenues = np.array([1000, 800, 600, 400])

    T = 400
    times = np.arange(T)

    L = 1
    customer_segments = np.arange(L)
    arrival_probabilities = np.array([0.5])
    preference_weights = np.array([[0.4, 0.8, 1.2, 1.6]])

    varNoPurchasePreferences = np.array([1, 2, 3])
    preference_no_purchase = np.array([varNoPurchasePreferences[0]])

    m = 1
    resources = np.arange(m)

    varCapacity = np.arange(40, 120, 20)
    capacities = np.array([varCapacity[0]])

    # capacity demand matrix A (rows: resources, cols: products)
    # a_ij = 1 if resource i is used by product j
    A = np.array([[1, 1, 1, 1]])

# %% Check up
print("Check of dimensions: \n  -------------------")
print("Ressourcen: \t", len(resources) == len(capacities) == A.shape[0])
print("Produkte: \t", len(products) == len(revenues) == preference_weights.shape[1] == A.shape[1])
print("Kundensgemente:\t", len(customer_segments) == len(arrival_probabilities) == preference_weights.shape[0] ==
      len(preference_no_purchase))

def get_data_for_table1():
    return resources, \
           products, revenues, A, \
           customer_segments, preference_weights, arrival_probabilities, \
           times

Check of dimensions: 
  -------------------
Ressourcen: 	 True
Produkte: 	 True
Kundensgemente:	 True


## Import der nötigen Bibliotheken

In [1]:
#  PACKAGES
# Data
import numpy as np
import pandas as pd

# Calculation and Counting
import itertools
import math

# Memoization
import functools

from dat_Koch import get_data_for_table1
from dat_Koch import get_variations

Check of dimensions: 
  ------------------------
Ressourcen: 	 True
Produkte: 		 True
Kundensgemente:	 True


In [1]:
from func_Koch import *
import inspect

Check of dimensions: 
  ------------------------
Ressourcen: 	 True
Produkte: 		 True
Kundensgemente:	 True


## Hilfsmethoden

Zunächst definiere ich einige Hilfsmethoden.

* memoize: Dient der späteren Memoisierung von Methoden (was zu beeindruckender Performanzverbesserung führt).
* offer_sets(products): Generates all possible offer sets.

In [2]:
lines = inspect.getsource(memoize)
print(lines)

def memoize(func):
    cache = func.cache = {}

    @functools.wraps(func)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return memoizer



In [13]:
# %% HELPER-FUNCTIONS
def memoize(func):
    cache = func.cache = {}

    @functools.wraps(func)

    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return memoizer

def offer_sets(products):
    """
    Generates all possible offer sets.

    :param products:
    :return:
    """
    n = len(products)
    offer_sets_all = np.array(list(map(list, itertools.product([0, 1], repeat=n))))
    offer_sets_all = offer_sets_all[1:]  # always at least one product to offer
    return offer_sets_all

## Funktionen

Hier folgen die eigentlichen Funktionen.
* customer_choice_individual(offer_set_tuple, preference_weights, preference_no_purchase): 
For one customer of one customer segment, determine its purchase probabilities given one offer set.
* customer_choice_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
From perspective of retailer: With which probability can he expect to sell each product (respectively non-purchase)

Die folgenden Funktionen sind in einer Art und Weise implementiert, sodass sie bspw. Table 1 nachbauen können und dennoch wenige Parameter in der memoize-Methode abgespeichert werden müssen (also Kapazitäten, Präferenzen für Nicht-Kauf und Zeit t als Parameter).
* def delta_value_j(j, capacities, t, preference_no_purchase):
For one product j, what is the difference in the value function if we sell one product.
* def value_expected(capacities, t, preference_no_purchase):
Recursive implementation of the value function, i.e. dynamic program (DP) as described on p. 241.

### customer_choice_individual

Gegeben sei ein Offerset: *x* $\in \{0,1\}^n$. 

Ein Kunde aus *Kundensegment l* mit Präferenz von $u_{lj}$ für *Produkt j* und Nichtkaufpräferenz von $u_{l0}$ erwirbt  Produkt j mit Wkeit

$p_{lj}(x) = \frac{u_{lj}x_j}{u_{l0} + \sum_{p\in[n]} u_{lp}x_p}$ .

Die Nichtkaufwahrscheinlichkeit beträgt dementsprechend

$p_{l0}(x) = 1 - \sum_{p\in[n]} p_{lj}(x)$ .

### customer_choice_vector

Der Retailer legt lediglich das Offerset *x* fest und kann dann die folgende Käufe erwarten keinenicht Wkeiten, da Summe nicht 1 ergeben muss).

$p_j(x) = \sum_{l\in[L]}\lambda_l p_{lj}(x)$ .

Die Nichtkaufwahrscheinlichkeit beträgt entsprechend

$p_0(x) = 1 - \sum_{p\in[n]} p_j(x)$ .

In [4]:
# %% FUNCTIONS
@memoize
def customer_choice_individual(offer_set_tuple, preference_weights, preference_no_purchase):
    """
    For one customer of one customer segment, determine its purchase probabilities given one offer set.

    Tuple needed for memoization.

    :param offer_set_tuple: tuple with offered products indicated by 1=product offered
    :param preference_weights: preference weights of one customer
    :param preference_no_purchase: no purchase preference of one customer
    :return: array of purchase probabilities ending with no purchase
    """

    if offer_set_tuple is None:
        ret = np.zeros_like(preference_weights)
        return np.insert(ret, -1, 1)

    offer_set = np.asarray(offer_set_tuple)
    ret = preference_weights * offer_set
    ret = np.array(ret / (preference_no_purchase + sum(ret)))
    ret = np.insert(ret, -1, 1 - sum(ret))
    return ret

In [15]:
@memoize
def customer_choice_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    From perspective of retailer: With which probability can he expect to sell each product (respectively non-purchase)

    :param offer_set_tuple: tuple with offered products indicated by 1=product offered
    :param preference_weights: preference weights of all customers
    :param preference_no_purchase: preference for no purchase for all customers
    :param arrival_probabilities: vector with arrival probabilities of all customer segments
    :return: array with probabilities of purchase ending with no purchase
    TODO probabilities don't have to sum up to one?
    """
    probs = np.zeros(len(offer_set_tuple) + 1)
    for l in np.arange(len(preference_weights)):
        probs += arrival_probabilities[l] * customer_choice_individual(offer_set_tuple, preference_weights[l, :],
                                                                       preference_no_purchase[l])
    return probs

### delta_value_j

### value_expected



In [10]:
def delta_value_j(j, capacities, t, A, preference_no_purchase):
    """
    For one product j, what is the difference in the value function if we sell one product.

    :param j:
    :param capacities:
    :param t:
    :param preference_no_purchase:
    :return:
    """
    return value_expected(capacities, t, preference_no_purchase)[0] - \
        value_expected(capacities - A[:, j], t, preference_no_purchase)[0]

In [11]:
@memoize
def value_expected(capacities, t, preference_no_purchase):
    """
    Recursive implementation of the value function, i.e. dynamic program (DP) as described on p. 241.

    :param capacities:
    :param t: time to go (last chance for revenue is t=0)
    :param preference_no_purchase:
    :return: value to be expected and optimal policy
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_for_table1()
    T = len(times)

    offer_sets_to_test = offer_sets(products)

    max_index = 0
    max_val = 0

    if all(capacities == 0):
        return 0, None
    if any(capacities < 0):
        return -math.inf, None
    if t == T:
        return 0, None

    for offer_set_index in range(len(offer_sets_to_test)):
        offer_set = offer_sets_to_test[offer_set_index, :]
        probs = customer_choice_vector(tuple(offer_set), preference_weights, preference_no_purchase,
                                       arrival_probabilities)

        val = float(value_expected(capacities, t + 1, preference_no_purchase)[0])  # ohne "float" würde ein numpy array
        #  zurückgegeben, und das später (max_val = val) direkt auf val verknüpft (call by reference)
        for j in products:  # nett, da Nichtkaufalternative danach kommt und nicht betrachtet wird
            p = float(probs[j])
            if p > 0.0:
                value_delta_j = delta_value_j(j, capacities, t + 1, A, preference_no_purchase)
                val += p * (revenues[j] - value_delta_j)

        if val > max_val:
            max_index = offer_set_index
            max_val = val
    return max_val, tuple(offer_sets_to_test[max_index, :])

## Ein paar Ergebnisse

Die Ergebnisse sehen ganz brauchbar aus. Wenn sie auch nicht exakt mit denen der Table 1 übereinstimmen. Warum nicht? Beachte die Code-Feinheiten mit "float" in value_expected().

<span style="color:red"> Ergebnisse stimmen teilweise mit Table 1 in Sebastians Paper überein, teilweise nicht. Siehe auch Anmerkung im Paper</span>

In [3]:
dftable1 = pd.read_pickle("table1_DP.pkl")
dftable1

,c,u,DP
0,[40],[1],"(39995.18483814307, (1, 0, 0, 0))"
1,[40],[2],"(38013.67099976408, (1, 0, 0, 0))"
2,[40],[3],"(35952.031147207534, (1, 1, 0, 0))"
3,[60],[1],"(58454.753775844474, (1, 0, 0, 0))"
4,[60],[2],"(53227.23269230522, (1, 1, 0, 0))"
5,[60],[3],"(49977.054186201975, (1, 1, 0, 0))"
6,[80],[1],"(73126.21245972312, (1, 1, 0, 0))"
7,[80],[2],"(66170.9528230564, (1, 1, 0, 0))"
8,[80],[3],"(59969.77918696259, (1, 1, 1, 0))"
9,[100],[1],"(87233.33717875695, (1, 1, 0, 0))"


# Parallel Flight Discussion

Aufbauend auf dem Ansatz von Miranda und Bront habe ich deren *choice based deterministic linear program (CDLP)* (p. 775) implementiert und deren *dynamic programming decomposition (DPD)*. Hier beschreiben wir zunächst den Beispieldatensatz parallel flights, bevor wir die Implementation vorstellen und die Ergebnisse für Table 3 darlegen.

## Example Three Parallel Flights (PF)

In [ ]:
n = 6
    products = np.arange(n)
    revenues = np.array([400, 800, 500, 1000, 300, 600])

    T = 300
    times = np.arange(T)

    L = 4
    customer_segments = np.arange(L)
    arrival_probabilities = np.array([0.1, 0.15, 0.2, 0.05])
    preference_weights = np.array([[0, 5, 0, 10, 0, 1],
                                   [5, 0, 1, 0, 10, 0],
                                   [10, 8, 6, 4, 3, 1],
                                   [8, 10, 4, 6, 1, 3]])

    var_no_purchase_preferences = np.array([[1, 5, 5, 1],
                                            [1, 10, 5, 1],
                                            [5, 20, 10, 5]])
    preference_no_purchase = var_no_purchase_preferences[0]

    m = 3
    resources = np.arange(m)

    base_capacity = np.array([30, 50, 40])
    delta = np.arange(0.4, 1.21, 0.2)
    var_capacities = np.zeros((len(delta), len(base_capacity)))
    for i in np.arange(len(delta)):
        var_capacities[i] = delta[i]*base_capacity
    capacities = var_capacities[0]

    # capacity demand matrix A (rows: resources, cols: products)
    # a_ij = 1 if resource i is used by product j
    A = np.array([[1, 1, 0, 0, 0, 0],
                  [0, 0, 1, 1, 0, 0],
                  [0, 0, 0, 0, 1, 1]])

## Funktionen

Die folgenden Inhalte sind sehr stark an Bront et al orientiert, wobei ich einige <span style="color:red"> Details farblich markiere. </span>

### Hilfsfunktionen
Sei S ein Offerset, also $x \in \{0,1\}^n$ mit $x_j=1$ nur falls j im Offerset. 

$P_{lj}(S)$ sei die Wahrscheinlichkeit, dass ein Kunde aus Segment $l$ Produkt $j$ kauft, wenn Offerset $S$ angeboten wird. Nach dem *multinomial logit model (MNL)* gilt:
$$ P_{lj}(S) = \frac{u_{lj}x_j}{\sum_{p \in S} u_{lp} x_p + u_{l0}}~.$$

Hieraus folgt, dass <span style="color:red"> gegeben ein Kunde kommt </span> und Offerset S wird angeboten, die Verkaufswkeit für Produkt j sich errechnet als
$$ P_j(S) = \sum_{l \in [L]} p_l P_{lj}(S) ~,$$
woraus sich $P(S) = (P_1(S), ..., P_n(S))^T$ ergibt.

<span style="color:red"> So wie wir die arrival probabilities $\lambda_l$ abspeichern, müssen wir hier $p_l$ durch Normierung berechnen. </span>

Dann berechnet sich der erwartete Erlös für ein gegebenes Offerset als
$$ R(S) = \sum_{j \in S} r_j P_j(S)~.$$

Gegeben die Ankunft eines Kunden, dann sei $Q_i(S)$ die bedingte Wkeit, dass eine Einheit der Ressource $i$ genutzt wird, falls $S$ angeboten wird. Sie berechnet sich als
$$ Q(S) = A P(S)~.$$

In [ ]:
def purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    P_j(S) for all j, P_0(S) at the end

    :param offer_set_tuple: S
    :param preference_weights
    :param preference_no_purchase
    :param arrival_probabilities
    :return: P_j(S) for all j, P_0(S) at the end
    """
    probs = np.zeros(len(offer_set_tuple) + 1)
    p = arrival_probabilities/(sum(arrival_probabilities))
    for l in np.arange(len(preference_weights)):
        probs += p[l] * customer_choice_individual(offer_set_tuple, preference_weights[l, :],
                                                   preference_no_purchase[l])
    return probs

In [ ]:
def revenue(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    R(S)

    :param offer_set_tuple: S
    :return: R(S)
    """
    return sum(revenues * purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase,
                                               arrival_probabilities)[:-1])

In [ ]:
def quantity_i(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities, i):
    """
    Q_i(S)

    :param offer_set_tuple: S
    :param i: resource i
    :return: Q_i(S)
    """
    return sum(A[i, :] * purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase,
                                              arrival_probabilities)[:-1])

### CDLP-base version

Die folgende Implementierung nimmt die gesamte Probleminstanz als Parameter und zusätzlich die zu betrachtenden Offersets. Es soll folgendes Optimierungsproblem gelöst werden.

\begin{align}
V^{CDLP} = \max & \sum_{S\in N} \lambda R(S) t(S) \\
s.t. & \sum_{S\in N} \lambda Q(S) t(S) \leq c~, \\
& \sum_{S\in N} t(S) \leq T~,\\
& t(S) \geq 0 \quad \forall S \in N~.
\end{align}